# OpenAI GPT Models Tutorial

This notebook covers working with OpenAI's GPT models using the `llm_playbook` package.

## What You'll Learn

- Setting up the OpenAI client
- Basic chat completions
- System prompts and message roles
- Multi-turn conversations
- Streaming responses
- Generation parameters (temperature, max_tokens)
- Response inspection (usage, model, finish_reason)

## Available Models

| Model | Description |
|-------|-------------|
| `gpt-4o` | Latest flagship model, multimodal |
| `gpt-4o-mini` | Fast and affordable (default) |
| `gpt-4-turbo` | Previous generation flagship |
| `gpt-4` | Original GPT-4 |

## Setup

First, install the package and configure your API key.

In [ ]:
# Install the package
!pip install -q git+https://github.com/deepakdeo/python-llm-playbook.git

In [ ]:
# Setup API Key from Colab Secrets
import os
from google.colab import userdata

# Add your OPENAI_API_KEY in the Secrets pane (🔑 icon in left sidebar)
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
print("API key configured!")

## 1. Basic Chat Completions

The simplest way to use OpenAI - just send a message and get a response.

In [ ]:
from llm_playbook import OpenAIClient

# Initialize the client (uses gpt-4o-mini by default)
client = OpenAIClient()

# Simple chat
response = client.chat("What is machine learning in one sentence?")
print(response)

In [ ]:
# Use a specific model
client_gpt4 = OpenAIClient(model="gpt-4o")

response = client_gpt4.chat("What makes GPT-4 different from GPT-3?")
print(response)

## 2. System Prompts and Message Roles

System prompts set the AI's behavior and personality. They're powerful for controlling responses.

In [ ]:
# Without system prompt
response = client.chat("Explain what an API is")
print("Without system prompt:")
print(response)
print()

In [ ]:
# With system prompt - as a teacher
response = client.chat(
    message="Explain what an API is",
    system_prompt="You are a friendly teacher explaining concepts to a 10-year-old. Use simple words and fun analogies."
)
print("As a teacher for kids:")
print(response)

In [ ]:
# With system prompt - as a technical expert
response = client.chat(
    message="Explain what an API is",
    system_prompt="You are a senior software architect. Be precise and use proper technical terminology."
)
print("As a technical expert:")
print(response)

## 3. Multi-turn Conversations

Maintain context across multiple exchanges using conversation history.

In [ ]:
from llm_playbook import ChatMessage

# Initialize conversation history
history = []
system_prompt = "You are a helpful astronomy expert. Keep answers concise."

# Turn 1
user_msg_1 = "What's the closest star to Earth?"
response_1 = client.chat(user_msg_1, system_prompt=system_prompt, history=history)

print(f"User: {user_msg_1}")
print(f"Assistant: {response_1}\n")

# Add to history
history.append(ChatMessage(role="user", content=user_msg_1))
history.append(ChatMessage(role="assistant", content=response_1))

In [ ]:
# Turn 2 - follows up on context
user_msg_2 = "Does it have any planets?"
response_2 = client.chat(user_msg_2, system_prompt=system_prompt, history=history)

print(f"User: {user_msg_2}")
print(f"Assistant: {response_2}\n")

# Add to history
history.append(ChatMessage(role="user", content=user_msg_2))
history.append(ChatMessage(role="assistant", content=response_2))

In [ ]:
# Turn 3 - continues the context
user_msg_3 = "Could humans ever travel there?"
response_3 = client.chat(user_msg_3, system_prompt=system_prompt, history=history)

print(f"User: {user_msg_3}")
print(f"Assistant: {response_3}")

## 4. Streaming Responses

Stream tokens as they're generated for real-time output. Great for chat interfaces!

In [ ]:
print("Streaming response: ", end="")

for token in client.stream("Write a haiku about programming."):
    print(token, end="", flush=True)

print()  # newline at end

In [ ]:
# Streaming with system prompt
print("Streaming with persona: ", end="")

for token in client.stream(
    message="Tell me a joke about Python",
    system_prompt="You are a stand-up comedian who loves programming humor."
):
    print(token, end="", flush=True)

print()

## 5. Generation Parameters

Control the output with parameters like `temperature` and `max_tokens`.

In [ ]:
# Low temperature (0.0) - deterministic, focused
response_low = client.chat(
    message="Give me a creative name for a coffee shop",
    temperature=0.0
)
print(f"Temperature 0.0: {response_low}")

In [ ]:
# High temperature (1.5) - more creative, varied
response_high = client.chat(
    message="Give me a creative name for a coffee shop",
    temperature=1.5
)
print(f"Temperature 1.5: {response_high}")

In [ ]:
# Limiting output length with max_tokens
response_short = client.chat(
    message="Explain the theory of relativity",
    max_tokens=50
)
print(f"Limited to 50 tokens:\n{response_short}")

In [ ]:
# Combining parameters
response = client.chat(
    message="Write a product description for a smart water bottle",
    system_prompt="You are a marketing copywriter. Be enthusiastic but concise.",
    temperature=0.7,
    max_tokens=100
)
print(response)

## 6. Response Inspection

Get detailed information about the response including token usage and metadata.

In [ ]:
# Get detailed response
response = client.chat_with_details(
    message="What is Python?",
    max_tokens=100
)

print("=== Response Details ===")
print(f"Content: {response.content}")
print(f"\nModel: {response.model}")
print(f"Finish reason: {response.finish_reason}")
print(f"\nToken usage:")
print(f"  - Prompt tokens: {response.usage['prompt_tokens']}")
print(f"  - Completion tokens: {response.usage['completion_tokens']}")
print(f"  - Total tokens: {response.usage['total_tokens']}")

In [ ]:
# Finish reason examples
# 'stop' = natural completion
# 'length' = hit max_tokens limit

# This will hit the length limit
response = client.chat_with_details(
    message="Write a 500-word essay about climate change",
    max_tokens=20
)

print(f"Content: {response.content}...")
print(f"Finish reason: {response.finish_reason}")
print("(Response was cut off due to max_tokens limit)")

## 7. Best Practices

Tips for working effectively with OpenAI's API.

In [ ]:
# Tip 1: Use clear, specific prompts
# Bad: "Write something about dogs"
# Good: "Write 3 bullet points about the health benefits of owning a dog"

response = client.chat(
    "Write 3 bullet points about the health benefits of owning a dog",
    max_tokens=150
)
print(response)

In [ ]:
# Tip 2: Use system prompts for consistent behavior
json_assistant = OpenAIClient()

response = json_assistant.chat(
    message="List 3 programming languages",
    system_prompt="You always respond in valid JSON format. No markdown, just pure JSON.",
    temperature=0.0
)
print(response)

In [ ]:
# Tip 3: Handle potential errors
try:
    response = client.chat("Hello!")
    print(f"Success: {response}")
except Exception as e:
    print(f"Error: {e}")

## Summary

You've learned:

1. **Basic usage**: `client.chat(message)` for simple queries
2. **System prompts**: Control behavior with `system_prompt` parameter
3. **Multi-turn**: Maintain context with `history` parameter
4. **Streaming**: Real-time output with `client.stream()`
5. **Parameters**: Control output with `temperature` and `max_tokens`
6. **Inspection**: Get metadata with `client.chat_with_details()`

## Next Steps

- Try the [Anthropic notebook](02_anthropic.ipynb) to compare with Claude
- Check out [06_comparison.ipynb](06_comparison.ipynb) for side-by-side comparisons
- Explore the [examples/](../examples/) directory for more patterns